In [5]:

import pandas as pd
import numpy as np
import math
import glob
import random
from collections import Counter
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag

import spacy
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec
from gensim import corpora


import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hamza.ahmed/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hamza.ahmed/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hamza.ahmed/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hamza.ahmed/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/hamza.ahmed/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [6]:

Data = pd.read_csv('/Users/hamza.ahmed/Python Apps/Data/Youtube_Knife_Reviews.csv')
Data.dropna()

,Channel,Video Title,Video ID,Video Likes,Video Length,Video Description,Video Tags,Video Transcript,Comment
0,Best Damn EDC [Taylor Martin],The BEST One-and-Done EDC Knife,Tfcu-GoZBeA,7843,0:15:57,Subscribe to my second channel https://bit.ly/...,"('EDC', 'everyday carry', 'edc weekly', 'edc g...",if you could have only one knife in your colle...,"I love my Civivis, but if I had to pick a ""one..."
1,Blade HQ,2021 Pocket Knife Holiday Gift Guide! | Knife ...,2_g-SdpHEWY,1136,0:18:55,Need a pocket knife as a gift for your loved o...,"('pocket knife', 'Blade HQ', 'BladeHQ', 'Knife...","How's it going, guys? Welcome to the 2021 gif...",Finally picked up a benchmade bug out after ye...
2,Blade HQ,The Best Kershaw Pocket Knives in 2020 at Blad...,qdbNejPUtUQ,2584,0:22:14,Zac and Jamie sit down at an appropriate dista...,"('Blade HQ', 'KnifeLife', 'KnifeBanter', 'Knif...",today a knife banter we're gonna talk\nkershaw...,Would love to see all the launch’s I would lov...
3,Blade HQ,The Best Pocket Knives of 2020 | Knife Banter ...,FUufroRWvfc,5785,0:37:54,Zac and Kurt run through the best knives (acco...,"('Blade HQ', 'BladeHQ', 'Knife Show', 'Sharp',...",today on knife banner we are talking the best ...,LOVE IT when you guys index these long videos....
4,Everyday Carry,The 8 Best Pocket Knives for EDC in 2021,GSejQk4gFcc,210,0:6:17,In today's video we'll be taking a look at our...,"('#Everydaycarry\u200b', '#Pocket\u200b', '#Kn...",if there's one item that separates your usual ...,I've been wracking my brain trying to find the...
5,KnifeCenter,Best CIVIVI Pocket Knives for EDC of 2020 - Kn...,GEYcxRvLw3g,1428,0:10:42,CIVIVI (the budget subsidiary of We Knife Co) ...,"('KnifeCenter', 'Knife Center', 'Knifecenter.c...",Hey everyone David C Andersen here\ncoming at ...,Civivi will be taking most of my paycheck with...
6,KnifeCenter,Best EDC Knives for 5th Pocket Carry of 2021 -...,ZhyJj8XMnPY,2738,0:17:51,"Whether you call it a watch pocket, a coin poc...","('KnifeCenter', 'Knife Center', 'Knifecenter.c...","Hey everyone, David C. Andersen here coming at...",The last year or two I’ve enjoyed carrying the...
7,KnifeCenter,Best Keychain Knives of 2021,54YumGPjO7Q,1805,0:25:13,"Whether for you or as a gift for someone else,...","('KnifeCenter', 'Knife Center', 'Knifecenter.c...","Hey everyone, David C Andersen here coming at\...",The pocket clip on the second knife he shows i...
8,KnifeCenter,"Best Knife from Every Brand in 2021, Part 1",cQOVB5UrOjc,7120,0:28:7,"So many companies make so many great knives, i...","('KnifeCenter', 'Knife Center', 'Knifecenter.c...","Hey everyone, David C Andersen here coming\nat...","David, you are unbelievably good in these vide..."
9,KnifeCenter,Best Pocket Cleaver Folding Knives of 2020 Ava...,8EYNe-Trgqs,1537,0:12:4,Pocket cleavers are some of the most awe-inspi...,"('KnifeCenter', 'Knife Center', 'Knifecenter.c...","Hey, everyone, David C. Andersen here coming\n...",This video was so incredibly well done and to ...


screen shot each video making grid
methodology
analysis
brand mentions
attributes
features
blade materials
categories ranked

In [10]:
Comment_Data = list(comment_words.lower() for comment_words in Data["Comment"])
#Tag_Data = list(tag_words.lower() for tag_words in Data["Video Tags"])
Transcribe_data = list(Transcribe_data.lower() for Transcribe_data in Data["Video Transcript"])
Description_data = list(Description_data.lower() for Description_data in Data["Video Description"])

In [389]:

# Knife_Companies =['benchmade', 'crkt', 'kershaw', 'chris reeves', 'chris reeve', 'civivi', "we knife"]

# Blade_Material=["AUS 4" , "AUS 8", "Bohler M390 ", "Bohler N680 ", "Bohler N690" , "M390" , "N680" , 
#                 "N690" , "Bohler" , "Böhler","CPM 154", "CPM 20CV" ,"CPM 3V " ,"CPM CruWear " , 
#                 "CPM D2", "CPM M4" ,"CPM S30V" ,"CPM S35VN" ,"CPM S45VN" ,"CPM S90V","CPM"]

# Handle Material=["aluminum", "brass", "carbon fiber" , "carbonfiber" , "forged carbon", "copper", "g10", "grivory", "micarta", "plastic", 
      #"stainless steel", "ss" "stainlesssteel", "Titanium"]
#Pivot_Assembly=["brass washer" , "BrassWasher", "Bronze washers" , "Bronzewashers", "Caged Ceramic", "Ball Bearing", "Caged Double" ,"Row Ceramic", "Caged Steel", "IKBS", "Phosphorous Bronze Washer" , "Phosphorous Bronze", "PhosphorousBronzeWasher","Teflon Washer"]
#Blade_Types=["Alloy", "Carbon Steel" , "CarbonSteel","D2", "Damasteel", "High Carbon", "HighCarbonStainlessSteel", "SS", "Stainless Steel" , "StainlessSteel" , "Stainless-Steel"]

#locking_mechanism = ["Back Lock","backlock",	"Button Lock","Buttonlock","Deadbolt", "Double Detent", "Slip Joint","Frame Lock",	"Front Lock","Frontlock", "Kinematic"	"Liner Lock",	"Linerlock", 	"Lock Back",	"lockback"]



In [28]:

#List=["aluminum", "brass", "carbon fiber" , "carbonfiber" , "forged carbon", "copper", "g10", "grivory", "micarta", "plastic", 
      #"stainless steel", "ss" "stainlesssteel", "Titanium"]
List = ["AUS 4" , "AUS 8", "Bohler M390 ", "Bohler N680 ", "Bohler N690" , "M390" , "N680" , 
"N690" , "Bohler" , "Böhler","CPM 154", "CPM 20CV" ,"CPM 3V " ,"CPM CruWear " , 
"CPM D2", "CPM M4" ,"CPM S30V" ,"CPM S35VN" ,"CPM S45VN" ,"CPM S90V","CPM","Damasteel", "High Carbon", 
"HighCarbonStainlessSteel", "SS", "Steel",  "Stainless Steel" , 
"StainlessSteel" , "Stainless-Steel", "Alloy", "Carbon Steel" , "CarbonSteel","D2"]



 
List = list(List.lower() for List in List)
########
#Tag Data
#######
#Vectorize the Data
# vectForKnifeCompanies = CountVectorizer(vocabulary=Knife_Companies, ngram_range=(1, 2))
# TD = vectForKnifeCompanies.fit_transform(Tag_Data)
# TD = TD.toarray()
# vectorizer_feature_names_Knife_Companies = vectForKnifeCompanies.get_feature_names_out()


# #Arrange
# Tag_Data = pd.DataFrame(data = TD, columns = vectorizer_feature_names_Knife_Companies)
# Tag_Data_For_Knife_Companies = Tag_Data.rename(columns={col: col+'_in_Tags' for col in Tag_Data.columns})
# print(Tag_Data_For_Knife_Companies)





vect = CountVectorizer(vocabulary=List, ngram_range=(1, 2))


########
#Description Data
#######

DD = vect.fit_transform(Description_data)
DD = DD.toarray()
vectorizer_feature_names = vect.get_feature_names_out()

DescriptionData = pd.DataFrame(data = DD, columns = vectorizer_feature_names)
Description_data_total = DescriptionData.rename(columns={col: col+'_in_Description' for col in DescriptionData.columns})
#print(Description_data_For_Knife_Companies )


########
#Transcript Data
#######


TRD = vect.fit_transform(Transcribe_data)
TRD = TRD.toarray()
vectorizer_feature_names = vect.get_feature_names_out()

TranscribeData = pd.DataFrame(data = TRD, columns = vectorizer_feature_names)
TranscribeData_total  = TranscribeData.rename(columns={col: col+'_in_Transcript' for col in TranscribeData.columns})
# print(TranscribeData_For_Knife_Companies)



########
#Comment Data
#######


CD = vect.fit_transform(Comment_Data)
CD = CD.toarray()
vectorizer_feature_names = vect.get_feature_names_out()


CommentData = pd.DataFrame(data = CD, columns = vectorizer_feature_names)
CommentData_total = CommentData.rename(columns={col: col+'_comments' for col in CommentData.columns})
# print(CommentData_For_Knife_Companies)



#Collecting it all!
Youtube_Knife_Data = pd.ExcelWriter('Youtube_Knife_Data.xlsx')

CommentData_total.to_excel(Youtube_Knife_Data, sheet_name='Comments')
TranscribeData_total.to_excel(Youtube_Knife_Data, sheet_name='Transcribe')
Description_data_total.to_excel(Youtube_Knife_Data, sheet_name='Description')

Youtube_Knife_Data.save()

In [ ]:
# #Knife Companies
# Knife_Companies =['benchmade', 'crkt', 'kershaw', 'chris reeves', 'chris reeve', 'civivi', "we knife"]

# ########
# #Tag Data
# #######
# #Vectorize the Data
# # vectForKnifeCompanies = CountVectorizer(vocabulary=Knife_Companies, ngram_range=(1, 2))
# # TD = vectForKnifeCompanies.fit_transform(Tag_Data)
# # TD = TD.toarray()
# # vectorizer_feature_names_Knife_Companies = vectForKnifeCompanies.get_feature_names_out()


# # #Arrange
# # Tag_Data = pd.DataFrame(data = TD, columns = vectorizer_feature_names_Knife_Companies)
# # Tag_Data_For_Knife_Companies = Tag_Data.rename(columns={col: col+'_in_Tags' for col in Tag_Data.columns})
# # print(Tag_Data_For_Knife_Companies)



# vectForKnifeCompanies = CountVectorizer(vocabulary=Knife_Companies, ngram_range=(1, 2))


# ########
# #Description Data
# #######

# DD = vectForKnifeCompanies.fit_transform(Description_data)
# DD = DD.toarray()
# vectorizer_feature_names_Knife_Companies = vectForKnifeCompanies.get_feature_names_out()

# DescriptionData = pd.DataFrame(data = DD, columns = vectorizer_feature_names_Knife_Companies)
# Description_data_For_Knife_Companies = DescriptionData.rename(columns={col: col+'_in_Description' for col in DescriptionData.columns})
# #print(Description_data_For_Knife_Companies )


# ########
# #Transcript Data
# #######


# TRD = vectForKnifeCompanies.fit_transform(Transcribe_data)
# TRD = TRD.toarray()
# vectorizer_feature_names_Knife_Companies = vectForKnifeCompanies.get_feature_names_out()

# TranscribeData = pd.DataFrame(data = TRD, columns = vectorizer_feature_names_Knife_Companies)
# TranscribeData_For_Knife_Companies = TranscribeData.rename(columns={col: col+'_in_Transcript' for col in TranscribeData.columns})
# # print(TranscribeData_For_Knife_Companies)



# ########
# #Comment Data
# #######


# CD = vectForKnifeCompanies.fit_transform(Comment_Data)
# CD = CD.toarray()
# vectorizer_feature_names_Knife_Companies = vectForKnifeCompanies.get_feature_names_out()


# CommentData = pd.DataFrame(data = CD, columns = vectorizer_feature_names_Knife_Companies)
# CommentData_For_Knife_Companies = CommentData.rename(columns={col: col+'_comments' for col in CommentData.columns})
# # print(CommentData_For_Knife_Companies)



# #Collecting it all!
# All_Knife_Company_Mentions = pd.ExcelWriter('All_Knife_Company_Mentions.xlsx')
# CommentData_For_Knife_Companies.to_excel(All_Knife_Company_Mentions, sheet_name='Comment Mentions')
# TranscribeData_For_Knife_Companies.to_excel(All_Knife_Company_Mentions, sheet_name='Transcribe Mentions')
# Description_data_For_Knife_Companies.to_excel(All_Knife_Company_Mentions, sheet_name='Description Mentions')

# All_Knife_Company_Mentions.save()

In [167]:
# count_vect = CountVectorizer(max_df=0.8, min_df=2, stop_words='english')
# doc_term_matrix = count_vect.fit_transform(Data["Video Transcript"].values.astype('U'))


In [168]:
# doc_term_matrix 

<1825x4657 sparse matrix of type '<class 'numpy.int64'>'
	with 999587 stored elements in Compressed Sparse Row format>

In [170]:
# LDA = LatentDirichletAllocation(n_components=5, random_state=42)
# LDA.fit(doc_term_matrix)

LatentDirichletAllocation(n_components=5, random_state=42)

In [176]:
# for i in range(10):
#     random_id = random.randint(0,len(count_vect.get_feature_names()))
#     print(count_vect.get_feature_names()[random_id])

uncomfortable
josh
water
initial
chrome
projects
arius
williamson
bolt
file


In [183]:
# first_topic = LDA.components_[0]
# top_topic_words = first_topic.argsort()[-10:]
# for i in top_topic_words:
#     print(count_vect.get_feature_names()[i])

pocket
tip
definitely
uh
sheath
gonna
carry
fixed
guy
um


In [184]:
# for i,topic in enumerate(LDA.components_):
#     print(f'Top 10 words for topic #{i}:')
#     print([count_vect.get_feature_names()[i] for i in topic.argsort()[-10:]])
#     print('\n')

Top 10 words for topic #0:
['pocket', 'tip', 'definitely', 'uh', 'sheath', 'gonna', 'carry', 'fixed', 'guy', 'um']


Top 10 words for topic #1:
['action', 'um', 'particular', 'pocket', 'version', 'design', 'different', 'shape', 'clip', 'lock']


Top 10 words for topic #2:
['military', 'carry', 'need', 'uh', 'look', 'cool', 'mean', 'gonna', 'guys', 'yeah']


Top 10 words for topic #3:
['ll', 'carbon', 'size', 'tang', 'bevel', 'cut', 'let', 'chef', 'survival', 'gonna']


Top 10 words for topic #4:
['automatic', 'lock', 'push', 'action', '2020', 'um', 'guys', 'cool', 'uh', 'yeah']




In [185]:
# topic_values = LDA.transform(doc_term_matrix)
# topic_values.shape

(1825, 5)

In [152]:
# stop = set(stopwords.words('english'))
# exclude = set(string.punctuation)
# lemma = WordNetLemmatizer()


# def clean(doc):
#     stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
#     punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
#     normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
#     return normalized


In [153]:
# doc_clean = [clean(doc).split() for doc in Data["Video Transcript"]]       

In [197]:
# x = Data['Video Transcript']
# keywords = ('knife', 'Sharpness', 'Sharp')
# if keywords in Data['Video Transcript']:
#     print (keywords)

0       today on knife banner we are talking the best ...
1       Hey everyone, David C. Andersen here coming\na...
2       hey everybody nakeeran today I got a review fo...
3       i'm will griffin i'm a knife maker the knife i...
4       hey everyone David C Andersen here\ncoming at ...
                              ...                        
1820    hey gang todd helms out here with eastman's \n...
1821    hey gang todd helms out here with eastman's \n...
1822    hey gang todd helms out here with eastman's \n...
1823    hey gang todd helms out here with eastman's \n...
1824    hey gang todd helms out here with eastman's \n...
Name: Video Transcript, Length: 1825, dtype: object

In [125]:

# #pd.DataFrame(zip(Data['Comment'],Data['Comment'].map(lambda d: re.escape("love") in d)))
# dictionary = corpora.Dictionary(doc_clean)
# doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [137]:
# Lda = gensim.models.ldamodel.LdaModel
# ldamodel = Lda(doc_term_matrix, num_topics=50, id2word = dictionary, passes=50)

In [149]:
#print(ldamodel.print_topics(num_topics=5, num_words=50))

In [ ]:
# search_list = ['poop']
# comm=Data.Comment.str.lower().str.split('\s')

# sq=Data.assign(c=comm.apply(lambda x:[*{*x}&{*search_list}])\
# .str.join(','),d=comm.apply(lambda \
# x:[*{*x}&{*search_list}]).str.len())


# # df1 = Data.Comment.str.split().apply(lambda x: Counter(x))
# # df1